In [1]:
import os
from datetime import datetime
from ltlcross_runner import LtlcrossRunner
from tools_hier import get_tools, ltl3dra_tools, full_tools, tool_order
from evaluation_utils import sort_by_tools, split_cols
import pandas as pd
from pandas.api.types import CategoricalDtype
import spot
spot.setup()

# Comparison of LTL to deterministic automata translators 

$\newcommand{\F}{\mathsf{F}}\newcommand{\G}{\mathsf{G}}\newcommand{\U}{\mathsf{U}}\newcommand{\X}{\mathsf{X}}$
## Tools
We have 
* tools that translate \[fragments\] of LTL into (generalized) Rabin automata:
 - [Rabinizer 3.1](https://www7.in.tum.de/~kretinsk/rabinizer3.html)
 - [Rabinizer 4](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - [LTL3DRA](https://sourceforge.net/projects/ltl3dra/) v.0.2.6 \[LTL$\smallsetminus\G$($\U\X$)\]
* tools that chain translation of LTL into cut-deterministic Büchi automata (aka LDBA) or into deterministic Rabin automata with a construction that create deterministic parity automaton. They avoid Safra-like determinization. Both are from the [owl/Rabinizer4 library](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - ltl2dpa --mode=ldba
 - ltl2dpa --mode=rabinizer (relies on transition-based index appearance record)
* tools that perform Safra-based determinization of N\[G\]BA into {Rabin/Parity} automata that use intermediete NBA
 - [ltl2dstar](http://ltl2dstar.de) \[NBA only\] {Rabin)}
 - autfilt from [Spot](https://spot.lrde.epita.fr/) 2.5 {parity}
 - here we use the following LTL2TGBA/LTL2NBA translators:
   - ltl2tgba from [Spot](https://spot.lrde.epita.fr/) 2.5
   - [LTL3BA](https://sourceforge.net/p/ltl3ba/) v. 1.1.3; we use LTL3BA in two settings:
     - `ltl3ba` {LTL3BA}
     - `ltl3ba -d` {LTL3BAd} which prefer more deterministic automata to smaller ones
 - we further use experimentaly for autfilt the LTL to Emerson-Lei automata translator
   - [LTL3TELA](https://github.com/jurajmajor/ltl3tela) v.1.1.1
* Safra's based determinization into deterministic parity automata that uses information about the LTL formula
 - `ltl2tgba -DG` from [Spot](https://spot.lrde.epita.fr/) 2.5

In [2]:
runners = {}
cols=["states","transitions","acc","time","nondet_states"]
for source in ('literature','random'):
    for t in ('full','ltl3dra'):
        name = '{}_{}'.format(source,t)
        tools = full_tools if t=='full' else ltl3dra_tools
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

name = 'random_fg'
tools = ltl3dra_tools
runners[name] = LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

In [3]:
for name, r in runners.items():
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    r.name = name
    r.parse_results()
    r.na_incorrect()
    r.orig_count = len(r.values)
    r.clean_count = len(r.values.dropna())

[31.01.2018 17:21:18]: Working on random_full
[31.01.2018 17:21:19]: Working on random_ltl3dra
[31.01.2018 17:21:19]: Working on random_fg
[31.01.2018 17:21:19]: Working on literature_ltl3dra
[31.01.2018 17:21:21]: Working on literature_full


In [4]:
for name, r in runners.items():
    print('The benchmark {} started with {} formulae and has valid data for {} formulae'.format(name,r.orig_count,r.clean_count))
    r.errors = {}
    for e in ['timeout', 'parse error', 'incorrect', 'crash', 'no output']:
        s = r.get_error_count(e).sum()
        if not pd.isna(s):
            r.errors[e] = r.get_error_count(e).sum()
    s = pd.Series(r.errors, name='count')
    s.index.name = 'error type'
    display(s.reset_index())

The benchmark random_full started with 500 formulae and has valid data for 479 formulae


,error type,count
0,parse error,27


The benchmark random_ltl3dra started with 498 formulae and has valid data for 492 formulae


,error type,count
0,crash,1
1,incorrect,2
2,parse error,6


The benchmark random_fg started with 488 formulae and has valid data for 487 formulae


,error type,count
0,incorrect,1


The benchmark literature_ltl3dra started with 92 formulae and has valid data for 89 formulae


,error type,count
0,parse error,2
1,timeout,3


The benchmark literature_full started with 129 formulae and has valid data for 121 formulae


,error type,count
0,parse error,5
1,timeout,5


## Gather data

In [5]:
def compute_runner(self,cols=['states','acc','time'],
                   tool_subset=None, split_col_names=True, col_lev_names=None,
                   counts = True):
    '''Returns a dataframe with rows indexed by tools in
    `tool_subset (or `self.tools.keys()`) and columns by
    cols. The values are cummulative sums for respective
    tool and column.'''
    if tool_subset is None:
        tool_subset = self.tools.keys()
    name = self.name
    name = name.replace('ltl3dra','ltl-gux')
    if counts:
        name = '{} ({})'.format(name,r.clean_count)
    col_names = ['{}_{}'.format(name,c) for c in cols]
    df = pd.DataFrame(columns=col_names,index=tool_subset)
    for col in cols:
        df['{}_{}'.format(name,col)] = self.cummulative(col).map(lambda x: '%0.0f' % x)
    if split_col_names:
        res = split_cols(df,axis=1,symbol='_',names=col_lev_names)
    else:
        res = df
    return res 

In [6]:
display(compute_runner(r).head())
display(compute_runner(r,split_col_names=False).head())
compute_runner(r,col_lev_names=['source','fragment','metric']).head()

literature          
                               full (121)          
                                   states  acc time
R3//DSRA                             1689  976  593
ltl2dstar/LTL3BA/NBA.DSRA            1988  396    6
ltl2dstar/Spot/NBA.DSRA              1095  336    5
Spot/Spot/TGBA.DTPA                   676  210    4
ltl2dstar(NBA)/LTL3BA/NBA.DSRA       2163  394    4

,literature_full (121)_states,literature_full (121)_acc,literature_full (121)_time
R3//DSRA,1689,976,593
ltl2dstar/LTL3BA/NBA.DSRA,1988,396,6
ltl2dstar/Spot/NBA.DSRA,1095,336,5
Spot/Spot/TGBA.DTPA,676,210,4
ltl2dstar(NBA)/LTL3BA/NBA.DSRA,2163,394,4


source                         literature          
fragment                       full (121)          
metric                             states  acc time
R3//DSRA                             1689  976  593
ltl2dstar/LTL3BA/NBA.DSRA            1988  396    6
ltl2dstar/Spot/NBA.DSRA              1095  336    5
Spot/Spot/TGBA.DTPA                   676  210    4
ltl2dstar(NBA)/LTL3BA/NBA.DSRA       2163  394    4

In [7]:
data = []
order = [
    ('literature','full'),
    ('literature','ltl3dra'),
    ('random','full'),
    ('random','ltl3dra'),
    ('random','fg'),
]
for o in order:
    data.append(compute_runner(runners['{}_{}'.format(o[0],o[1])]))
res = pd.concat(data,axis=1)
res = split_cols(res,axis=0,symbol='/')

In [8]:
sort_by_tools(res,tool_order)

literature                               \
                                   full (121)           ltl-gux (121)        
                                       states  acc time        states  acc   
LTL3DRA        -        DSRA              NaN  NaN  NaN           451  276   
                        DTGRA             NaN  NaN  NaN           315  172   
R3             -        DSRA             1689  976  593          1623  548   
                        DTGRA             851  425  427           870  191   
R4             -        DSRA             1323  410  271           384  220   
                        DTGRA             813  258  276           265  130   
ltl2dstar      LTL3BA   NBA.DSRA         1988  396    6         59569  276   
               LTL3BAd  NBA.DSRA         1228  374    4         59540  274   
               Spot     NBA.DSRA         1095  336    5         59750  266   
ltl2dstar(NBA) LTL3BA   NBA.DSRA         2163  394    4         78672  264   
               LTL3BAd  NBA.DSRA         1298  372    3         78681  264   
               Spot     NBA.DSRA         1135  330    4         79008  238   
Spot           LTL3BA   NBA.DTPA         1203  282    4           495  148   
                        TGBA.DTPA         979  249    3           452  152   
               LTL3BAd  NBA.DTPA          730  253    3           482  141   
                        TGBA.DTPA         698  254    3           437  148   
               LTL3TELA DTELA.DTPA        745  249  125           428  159   
               Spot     NBA.DTPA          680  212    3           448  127   
                        TGBA.DTPA         676  210    4           420  134   
ltl2tgba       -        DPA               672  206    2           420  134   
ltl2dpa        Rab      DTPA              876  347  274           411  202   
               ldba     DTPA              744  325  242           292  201   

                                            random                           \
                                        full (121)            ltl-gux (121)   
                                   time     states   acc time        states   
LTL3DRA        -        DSRA          3        NaN   NaN  NaN          3959   
                        DTGRA         2        NaN   NaN  NaN          3353   
R3             -        DSRA        155      10615  6072  449          4282   
                        DTGRA       139       4269  2839  268          2760   
R4             -        DSRA         58       5654  1626  892          2822   
                        DTGRA        60       3706  1091  892          2448   
ltl2dstar      LTL3BA   NBA.DSRA     24      39062  2008   24          4117   
               LTL3BAd  NBA.DSRA     23       9394  1824   19          4133   
               Spot     NBA.DSRA     24       8364  1720   22          4148   
ltl2dstar(NBA) LTL3BA   NBA.DSRA     26      49845  1956   16          4938   
               LTL3BAd  NBA.DSRA     26      11844  1772    9          4994   
               Spot     NBA.DSRA     26      10137  1672   13          4953   
Spot           LTL3BA   NBA.DTPA      1       4981  1146   13          2823   
                        TGBA.DTPA     1       3852  1026   10          2634   
               LTL3BAd  NBA.DTPA      1       4281  1049   10          2840   
                        TGBA.DTPA     1       3807  1052   11          2647   
               LTL3TELA DTELA.DTPA  122       4346  1071   18          2663   
               Spot     NBA.DTPA      1       4024   986   13          2711   
                        TGBA.DTPA     1       3697   984   14          2605   
ltl2tgba       -        DPA           1       3625   925    9          2600   
ltl2dpa        Rab      DTPA         66       4422  1324  952          2486   
               ldba     DTPA         55       6308  1295  870          2511   

                                                                   
                                        